# Job Application Log - JAL


The goal of this project is to allow for quickly saving the job description details associated with a job application and then to do analysis on the data collected.

This notebook compliments the main JAL.py dashboard - it is used for development and testing.


In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re
import panel as pn
import json
import pandas as pd
import datetime as dt
import altair as alt

pn.extension()


def get_directories():
    
    return_dict = {}
    jobs_directory = "jobs"
    return_dict['html'] = os.path.join(jobs_directory, 'html')
    return_dict['raw'] = os.path.join(jobs_directory, 'raw')
    return_dict['json'] = os.path.join(jobs_directory, 'json')
    
    if not os.path.exists(jobs_directory):
        os.mkdir(jobs_directory)

    if not os.path.exists(return_dict['html']):
        os.mkdir(return_dict['html'])
    
    if not os.path.exists(return_dict['raw']):
        os.mkdir(return_dict['raw'])
        
    if not os.path.exists(return_dict['json']):
        os.mkdir(return_dict['json'])
    return return_dict
        

directories = get_directories()


# Test LinkedIn Job Data Parser

"LinkedIn Job Details" are not available in the raw html file, so they are copy and pasted into the dashboard.  This code block is used to test the regular expressions.


In [79]:

test_data1 = """Principal Data Security Software Engineer
Roblox · San Mateo, CA (Hybrid) Reposted  2 weeks ago  · 29 applicants
$315,850/yr - $383,910/yr (from job description) · Full-time · Mid-Senior level
1,001-5,000 employees · Software Development
1 company alum works here
Skills: Cryptography, Application Security, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data2 = """Data Engineering Lead
Genentech · South San Francisco, CA  1 week ago  · 41 applicants
$172,500/yr - $320,300/yr (from job description) · Full-time · Associate
10,001+ employees · Biotechnology Research
Skills: Data Modeling, Data Architecture, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data3 = """Security Infrastructure DevOps Engineer, SEAR
Apple · Cupertino, CA Reposted  1 hour ago  · 238 applicants
$130,000/yr - $242,000/yr (from job description) · Full-time
10,001+ employees · Computers and Electronics Manufacturing
9 company alumni work here
Skills: Information Security Analysis, Independent Thinking, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data4 = """Lead software engineer (infrastructure)
AI Fund · San Francisco County, CA Reposted  1 hour ago  · 50 applicants
Full-time · Mid-Senior level
11-50 employees · Venture Capital and Private Equity Principals
Skills: Amazon Web Services (AWS), Optimization, +8 more
"""

test_data5 = """Service Team Lead II- Global Fix Experience
Uber · San Francisco, CA Reposted  2 days ago  · 41 applicants
$34.50/hr (from job description) · Full-time
10,001+ employees · Software Development
2 company alumni work here
Skills: Google Workspace, Query Writing, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data6 = """Manager Regulatory Compliance Audits
Intuitive · Sunnyvale, CA (On-site) Reposted  2 days ago  · 57 applicants
$118,500/yr - $200,600/yr · Full-time · Mid-Senior level
5,001-10,000 employees · Medical Equipment Manufacturing
Skills: Regulatory Reporting, Regulatory Requirements, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data7 = """Lead ML Infrastructure Engineer
DocuSign · San Francisco, CA (Hybrid) Reposted  2 weeks ago  · 16 applicants
Full-time
5,001-10,000 employees · Software Development
1 company alum works here
Skills: Reliability, Complex Systems, +7 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""

test_data8 = """Data Engineer, Product Analytics
Meta · United States (Remote) Reposted  2 weeks ago  · 280 applicants
$134,000/yr - $194,000/yr · Full-time
10,001+ employees · Software Development
8 company alumni work here
Skills: Data Engineering, Data Warehousing, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""
test_data9 = """Data Engineer, Data Platform
Grammarly · California, United States (Hybrid)  4 hours ago  · 3 applicants
$134,000/yr - $242,000/yr (from job description) · Full-time · Mid-Senior level
501-1,000 employees · Technology, Information and Internet
Skills: Data Engineering, Databases, +8 more
View verifications related to this job post.View verifications related to this job post.
Show all
"""


job_types = ['Full-time', 'Contract', 'Volunteer', 'Other', 'Part-time', 'Temporary', 'Internship']

# Contract Full-time
def parse_salary_range(salary_range):
    return_value = {}
    #salary_re = r"(\$[^\(]+) \(from job description\).*"
    salary_re = r"(\$[^\(]+).*"
    search_results = re.search(salary_re, salary_range)
    salary = search_results.group(1).strip()
    if salary.endswith('/hr'):
        pattern = r"\$(\d+\.\d+)/hr"
        match = re.match(pattern, salary)
        return_value['hourly_rate'] = float(match.group(1))
    else:
        pattern = r"\$(\d+?,\d+)/yr - \$(\d+?,\d+)/yr"
        match = re.match(pattern, salary)

        return_value['starting_salary_range'] = match.group(1).replace(",", "")
        return_value['ending_salary_range'] = match.group(2).replace(",", "")
    return return_value
    

def parse_job_details(job_details):
    
    lines = job_details.splitlines()
    job_data = {}
    
    # Line 1
    job_data['job_title'] = lines[0].strip()
    
    # Line 2
    line2_data = lines[1].split('·')
    job_data['company_name'] = line2_data[0].strip()
    if "Remote" in line2_data[1]:
        job_data['location'] = 'Remote'
    else:
        search_results = re.search(r"(.*, [A-Z]{2}) .*", line2_data[1])
        if search_results:
            job_data['location'] = search_results.group(1).strip()
        else:
            job_data['location'] = 'Remote'
    
    search_results = re.search(r"(\d+) applicants.*", line2_data[2])
    job_data['applicants'] = search_results.group(1).strip()
    
    # Line 3
    line3_data = lines[2].split('·')
    
    if len(line3_data) == 3:
        #job_data['salary_range'] = line3_data[0].strip()
        job_data['job_type'] = line3_data[1].strip()
        job_data['experience_level'] = line3_data[2].strip()
        job_data.update(parse_salary_range(line3_data[0].strip()))
    elif len(line3_data) == 1:
        if line3_data[0].strip() in job_types:
            job_data['job_type'] = line3_data[0].strip()
        else:
            job_data['experience_level'] = line3_data[0].strip()
    else:
        if line3_data[0].strip() in job_types:
            job_data['job_type'] = line3_data[0].strip()
            job_data['experience_level'] = line3_data[1].strip()
        else:
            job_data.update(parse_salary_range(line3_data[0].strip()))
            if line3_data[1].strip() in job_types:
                job_data['job_type'] = line3_data[1].strip()
            else:
                job_data['experience_level'] = line3_data[1].strip()
       
    # Line 4
    line4_data = lines[3].split('·')
    job_data['company_size'] = line4_data[0].strip()
    job_data['industry'] = line4_data[1].strip()
    
    return {"job": job_data}
    
job_data1 = parse_job_details(test_data1)
job_data2 = parse_job_details(test_data2)
job_data3 = parse_job_details(test_data3)
job_data4 = parse_job_details(test_data4)
job_data5 = parse_job_details(test_data5)
job_data6 = parse_job_details(test_data6)
job_data7 = parse_job_details(test_data7)
job_data8 = parse_job_details(test_data8)
job_data9 = parse_job_details(test_data9)
#print(json.dumps(job_data1, indent=4))

assert len(job_data1['job']) == 10
assert len(job_data2['job']) == 10
assert len(job_data3['job']) == 9
assert len(job_data4['job']) == 8
assert len(job_data5['job']) == 8
assert len(job_data6['job']) == 10
assert len(job_data7['job']) == 7
assert len(job_data8['job']) == 9
assert len(job_data9['job']) == 10

assert job_data5['job']['hourly_rate'] == 34.50
assert job_data2['job']['company_name'] == 'Genentech'

def materialize_job_details(linkedin_job_code, job_details):
    raw_filename = f"{linkedin_job_code}.txt"
            
    if not os.path.exists(os.path.join(directories["raw"], raw_filename)):
        with open(os.path.join(directories["raw"], raw_filename), "w") as f:
            f.write(job_details)
            f.close()

materialize_job_details('1234', test_data2)

assert os.path.exists(os.path.join(directories["raw"], '1234.txt'))
os.remove(os.path.join(directories["raw"], '1234.txt'))


## Compile Pandas DataFrame


In [6]:

def compile_dataframe(directory):
    df_columns=['job_code', 'job_title', 'company_name', 'location', 'starting_salary_range', 'ending_salary_range', 'experience_level', 'job_type', 'company_size', 'industry', 'application_date', 'applicants']
    df = pd.DataFrame(columns=df_columns)
    for file in os.listdir(directory):
        if file.endswith('.json'):
            with open(os.path.join(directory,  file)) as f:
                new_data = pd.read_json(os.path.join(directory,  file), orient='index')
                df = pd.concat([df, new_data])
    df.set_index('job_code', inplace=True, drop=False)
    df.to_json('JAL.json', orient='index', indent=6)
    return df
    

df = compile_dataframe('jobs/json')

### Test Initializing the Pandas DataFrame


In [7]:
df = pd.read_json('JAL.json', orient='index')
df.set_index("job_code", inplace=True, drop=False)
#['job_code', 'job_title', 'company_name', 'location', 'starting_salary_range', 'ending_salary_range', 'experience_level', 'job_type', 'company_size', 'industry', 'application_date', 'applicants']
    
df["application_date"] = pd.to_datetime(df["application_date"])
df["job_type"] = df["job_type"].astype("category")
df["company_size"] = df["company_size"].astype("category")
df["industry"] = df["industry"].astype("category")
df["job_code"] = df["job_code"].astype("str")

#tabulator = df[['job_code', 'job_title', 'company_name', 'application_date']]

#tabulator.head()

#df['application_date'].min()
#df['application_date'].dt.to_datetime()
#dt.date.fromtimestamp(df['application_date'].min())
df['application_date'].min().to_pydatetime()

datetime.datetime(2023, 6, 16, 0, 0)

### BeautifulSoup

Test block for parsing the html file


In [2]:


def beautiful_soup_test(url):
    jd = MaterializeJobData(url)
    soup = BeautifulSoup(jd, "html.parser")

    jt = soup.find("meta", {"property": "og:title"})
    company_name = re.search(r"(.*) hiring .+$", jt['content']).group(1)
    job_title = re.search(r"hiring (.*) in .+$", jt['content']).group(1)
    location = re.search(r"in (.*) \| .+$", jt['content']).group(1)

    job_data = {
        "job_title": job_title,
        "company_name": company_name,
        "location": location
    }
        
    return job_data


141896


In [44]:


# create a dataframe using uscities.csv. create a new field that combines city and state with a comma
# and merge with the dataframe above
cities = pd.read_csv('uscities.csv')
cities['city_state'] = cities['city'] + ', ' + cities['state_id']

# limit cities to only include cupertino, ca
#cities = cities[cities['city_state'].str.contains("Cupertino, CA")]

#cities.head()


df2 = pd.merge(df, cities, left_on='location', right_on="city_state", how='inner')
#df2.head(50)

# using altair, create a Scatter-Plots with Geographic Coordinates where the size of the circle is the number of applications

alt.Chart(df2).mark_circle().encode(
    longitude='lng_x:Q',
    latitude='lat_x:Q',
    #size=alt.value(10),
    tooltip='city_state'
).project(
    "albersUsa"
).properties(
    width=500,
    height=400
)

# using the following, create a version of the above df2 chart with a colored background: mark_geoshape(fill = "lightgray", stroke = "white")









alt.Chart(...)

In [8]:
from JAL import read_job_record, write_job_record

def load_json_files(directory):
    df_columns=['job_code', 'job_title', 'company_name', 'location', 'starting_salary_range', 'ending_salary_range', 'experience_level', 'job_type', 'company_size', 'industry', 'application_date', 'applicants']
    df = pd.DataFrame(columns=df_columns)
    for file in os.listdir(directory):
        if file.endswith('.json'):
            # print file name without extension
            job_code = os.path.splitext(file)[0]
            job = read_job_record(job_code)
            #job['job']['status'] = "open"
            job = write_job_record(job)
            #print(job['job']['status'])
    return df
    

df = load_json_files('jobs/json')